In [1]:
from buildingmotif import BuildingMOTIF
from buildingmotif.model_builder import TemplateBuilderContext
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes, UNIT
from rdflib import Namespace

# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://", shacl_engine="topquadrant")

In [2]:
# create the model w/ a namespace
BLDG = Namespace("urn:nrel_example/")
WATR = Namespace("urn:nawi-water-ontology#")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

s223 = Library.load(ontology_graph="../libraries/223p.ttl", infer_templates=False)
s223 = Library.load(ontology_graph="../libraries/water.ttl", infer_templates=False)
# nrel = Library.load(directory="../libraries/nrel-223p-templates")
water = Library.load(directory="../libraries/templates")

In [3]:
# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
# ctx.add_templates_from_library(nrel)
ctx.add_templates_from_library(water)

## Modeling a Tank with a Flow Sensor

Looking at our templates, we can see a [Tank template](https://datadrivencps.github.io/water-ontology/libraries/templates/tank.html) and a [Sensor template](https://datadrivencps.github.io/water-ontology/libraries/nrel-223p-templates/sensor.html). We will also use the [water flow](https://datadrivencps.github.io/water-ontology/libraries/nrel-223p-templates/water-flow.html) property template to model the actual property being measured by the sensor.

The tank has a few required parameters. It needs a name (like all entities), and either an inlet/outlet connection point pair OR a single bi-directional connection point.

A sensor  needs a name as well as a property it is observing, and a location where that property is observed.

In [4]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
my_tank = ctx['tank'](name='my_tank')

# now create an instance of the water flow property
flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
flow_prop['name-owner'] = my_tank['out']

In [5]:
# now create the sensor
flow_sensor = ctx['sensor'](name='my_flow_sensor')
flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [6]:
# concentration sensor
salt_concentration_sensor = ctx['concentration-sensor'](name='salt_conc_sensor', substance=WATR["Constituent-Salt"], unit=UNIT["PPM"])
salt_concentration_sensor['where'] = my_tank['out']

In [7]:
def pipe(in_=None, out_=None):
    p = ctx['pipe']
    p['in'] = in_
    p['out'] = out_
    return p

In [8]:
# variadic parameters with junctions
# tank1 --O--> my_tank
#         |
# tank2 --+
# where O is the junction
tank1 = ctx['tank'](name='tank1')
tank2 = ctx['tank'](name='tank2')
junction = ctx['junction'](name='my_tank_inlet_junction')
junction.to_variadic('in') # turns the 'in' parameter variadic

pipe(tank1['out'], junction['in'])
pipe(tank2['out'], junction['in'])

pipe(junction['out'], my_tank['in'])

In [9]:
bldg.add_graph(ctx.compile())
graph = bldg.compile([s223.get_shape_collection()])

/Users/gabe/src/NAWI/water-ontology/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:483: UserWarning: Parameters "property, property-owner, property-constituent" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NAWI/water-ontology/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:483: UserWarning: Parameters "medium, in" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NAWI/water-ontology/.venv/lib/python3.10/site-packages/buildingmotif/dataclasses/template.py:483: UserWarning: Parameters "name" were not provided during evaluation
  warnings.warn(


In [10]:
# this is the serialization of the model for machine-to-machine
print(graph.model.graph.serialize(format="turtle"))

@prefix bldg: <urn:nrel_example/> .
@prefix ns1: <http://data.ashrae.org/standard223#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix qudtqk: <http://qudt.org/vocab/quantitykind/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

bldg: a owl:Ontology .

bldg:my_flow_sensor a ns1:Sensor ;
    rdfs:label "Sensor" ;
    ns1:hasObservationLocation bldg:out_810ba45f ;
    ns1:observes bldg:tank_outlet_water_flow .

bldg:my_tank a <urn:nawi-water-ontology#Tank> ;
    rdfs:label "Tank" ;
    ns1:cnx bldg:in_e3bbd00e,
        bldg:out_810ba45f ;
    ns1:hasConnectionPoint bldg:in_e3bbd00e,
        bldg:out_810ba45f .

bldg:my_tank_inlet_junction a ns1:Junction ;
    rdfs:label "Junction" ;
    ns1:cnx bldg:in0_54e48fac,
        bldg:in1_cb91f542,
        bldg:out_931ca8d3 ;
    ns1:hasMedium bldg:medium_63f89a64 .

bldg:name_1fc5ef22 a ns1:Connection,
        ns1:Pipe ;
    rdfs:lab